In [17]:
import requests
from selenium import webdriver
import numpy as np
import pandas as pd
import datetime
import json
import pyodbc
import mysql.connector

URL_SEARCH = "https://www.tickpick.com/concerts/yeat-tickets/"

ModuleNotFoundError: No module named 'mysql'

In [2]:
driver = webdriver.Chrome(r"C:\Users\Paul\Documents\chromedriver.exe")
driver.get(URL_SEARCH)

xpath_links = "//*[@id=\"events\"]/div[*]/a[1]"


links = driver.find_elements_by_xpath(xpath_links)
venue_names = driver.find_elements_by_xpath("//*[@id=\"events\"]/div[*]/div[2]/span/span")
dates = driver.find_elements_by_xpath("//*[@id=\"events\"]/div[*]/div[1]/span[1]")

links = list(map(lambda elem: elem.get_attribute("href"), links))
venue_names = list(map(lambda elem: elem.text, venue_names))
dates = list(map(lambda elem: elem.text, dates))


In [3]:
def add_page_to_row_list(link, row_list, driver, venue_name, date, time):
    driver.get(link)

    xpath = "//*[@id=\"listingContainer\"]/div[*]"

    elements = []
    
    while len(elements) == 0:
        elements = driver.find_elements_by_xpath(xpath)

    for elem in elements:
        dic = {}
        dic["Date"] = time
        dic["VenueID"] = venue_name
        dic["DateEvent"] = date
        dic["Quantity"] = elem.find_element_by_tag_name("select").text.split()[0]
        dic["Price"] = elem.find_element_by_tag_name("b").text[1:]
        row_list.append(dic)

In [4]:
row_list = []

date = datetime.datetime.now()
time = date.strftime("%m.%d.%y_%H.%M")

for i in range(len(links)):
    add_page_to_row_list(links[i], row_list, driver, venue_names[i], dates[i], time)

df = pd.DataFrame(row_list)
df.head()
#date = datetime.datetime.now()
#df.to_csv(time, index = False)

,Date,VenueID,DateEvent,Quantity,Price
0,04.04.22_21.59.csv,"Concord Music Hall - Chicago, IL",Apr 07,2,
1,04.04.22_21.59.csv,"Concord Music Hall - Chicago, IL",Apr 07,2,
2,04.04.22_21.59.csv,"Concord Music Hall - Chicago, IL",Apr 07,2,
3,04.04.22_21.59.csv,"Concord Music Hall - Chicago, IL",Apr 07,4,
4,04.04.22_21.59.csv,"Concord Music Hall - Chicago, IL",Apr 07,6,


In [7]:
df["Price"].head(100)

0        
1        
2        
3        
4        
     ... 
95     99
96    100
97    102
98    102
99    104
Name: Price, Length: 100, dtype: object

In [16]:
server = 'tickpick-scraper-database.mysql.database.azure.com'
database = 'tickpick-scraper-database'
username = 'pauldoucet'

#cnx = mysql.connector.connect(user="pauldoucet", password="{your_password}", host="tickpick-scraper-database.mysql.database.azure.com", port=3306, database="{your_database}", ssl_ca="{ca-cert filename}", ssl_disabled=False)

f = open("password.json")
data = json.load(f)
password = data["password"]

driver= '{ODBC Driver 18 for SQL Server}'

with pyodbc.connect('DRIVER='+driver+';SERVER=tcp:'+server+';PORT=3306;DATABASE='+database+';UID='+username+';PWD='+ password) as conn:
    with conn.cursor() as cursor:
        cursor.execute("SELECT TOP 3 name, collation_name FROM sys.databases")
        row = cursor.fetchone()
        while row:
            print (str(row[0]) + " " + str(row[1]))
            row = cursor.fetchone()

OperationalError: ('08001', '[08001] [Microsoft][ODBC Driver 18 for SQL Server]TCP Provider: Dépassement du délai d’attente.\r\n (258) (SQLDriverConnect); [08001] [Microsoft][ODBC Driver 18 for SQL Server]Login timeout expired (0); [08001] [Microsoft][ODBC Driver 18 for SQL Server]Invalid connection string attribute (0); [08001] [Microsoft][ODBC Driver 18 for SQL Server]A network-related or instance-specific error has occurred while establishing a connection to SQL Server. Server is not found or not accessible. Check if instance name is correct and if SQL Server is configured to allow remote connections. For more information see SQL Server Books Online. (258)')